In [1]:
import csv
from collections import Counter
import pandas as pd
from math import log
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import gensim
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora

C:\Users\happy\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
year_list = ['2011', '2012', '2013','2014','2015','2016','2017','2018','2019','2020']

In [3]:
#tf-idf sklearn
for year in year_list:
    doc_list = list()
    txt = ''

    file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav2/'+year+'.csv','r',encoding='utf-8-sig')
    reader = csv.reader(file)

    for line in reader:
        word_list = list()
        word_list = [word.rstrip('\n') for word in line]
        txt = ' '.join(word_list)    
        doc_list.append(txt)
    
    file.close()

    vectorizer  = TfidfVectorizer(min_df = 2, analyzer = 'word')

    tfidfv = vectorizer.fit(doc_list)
    tfidfa = tfidfv.transform(doc_list).toarray()
    
    df = pd.DataFrame(tfidfa, columns = sorted(tfidfv.vocabulary_))
    
    lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=30)

    lda_top = lda_model.fit_transform(tfidfa)

    terms = vectorizer.get_feature_names()

    def get_topics(components, feature_names, n=5):
        for idx, topic in enumerate(components):
            tmp_list = list()
            tmp_list.append("Topic "+str(idx+1)+': ')
            tmp_list += ([(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
            print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
            with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav2/LDA/'+year+'.csv', 'a+', newline='', encoding='utf-8-sig') as f:
                writer = csv.writer(f)
                writer.writerow(tmp_list)

    get_topics(lda_model.components_,terms)

KeyboardInterrupt: 

In [15]:
#tf-idf gensim
for year in year_list:
    doc_list = list()
    file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav3/'+year+'.csv','r',encoding='utf-8-sig')
    reader = csv.reader(file)

    for line in reader:
        word_list = list()
        word_list = [word.rstrip('\n') for word in line if word != ""]   
        doc_list.append(word_list)
    
    file.close()

    dictionary = corpora.Dictionary(doc_list)
    corpus = [dictionary.doc2bow(text) for text in doc_list]

    NUM_TOPICS = 20 # 20개의 토픽, k=20
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=20, iterations = 100)
    topics = ldamodel.print_topics()
    
    
    
    with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav3/LDA/gen_'+year+'.csv', 'a+', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        for topic in topics:
            writer.writerow(topic)

In [3]:
#LDA gensim
for year in year_list:
    doc_list = list()
    file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav7/'+year+'.csv','r',encoding='utf-8-sig')
    reader = csv.reader(file)

    for line in reader:
        word_list = list()
        word_list = [word.rstrip('\n') for word in line if word != ""]   
        doc_list.append(word_list)
    
    file.close()

    dictionary = corpora.Dictionary(doc_list)
    dictionary.filter_extremes(no_below=10, no_above=0.07)
    corpus = [dictionary.doc2bow(text) for text in doc_list]

    NUM_TOPICS = 30 # 20개의 토픽, k=20
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30, iterations = 100)
    topics = ldamodel.print_topics(num_topics = NUM_TOPICS, num_words = 20)
    
    tmp1 = list()
    topics = list(topics)
    
    for t in topics:
        index_t = topics.index(t)
        tmp2 = t[1].split(' + ')
        for w in tmp2:
            tmp3 = w.split('*')
            tmp3[1] = tmp3[1].strip('"')
            if index_t == 0: 
                tmp1.append(tmp3)
            else:
                try:
                    tmp1[tmp2.index(w)] += tmp3
                except:
                    print(tmp2.index(w), len(tmp1), t[0], type(t[0]))

    with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav7/LDA/gen_'+year+'.csv', 'a+', newline = '', encoding='utf-8-sig') as ldafile:
        writer = csv.writer(ldafile)
        writer.writerows(tmp1)

In [ ]:
#tf-idf sklearn
doc_list = list()
txt = ''

file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav2/2011.csv','r',encoding='utf-8-sig')
reader = csv.reader(file)

for line in reader:
    word_list = list()
    word_list = [word.rstrip('\n') for word in line]
    txt = ' '.join(word_list)    
    doc_list.append(txt)

file.close()

vectorizer  = TfidfVectorizer(min_df = 2, analyzer = 'word')

tfidfv = vectorizer.fit(doc_list)
tfidfa = tfidfv.transform(doc_list).toarray()

df = pd.DataFrame(tfidfa, columns = sorted(tfidfv.vocabulary_))

lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=30)

lda_top = lda_model.fit_transform(tfidfa)

terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

In [47]:
#형식 맞추기
for year in year_list:
    t_list = list()
    with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav3/LDA/gen_'+year+'.csv', 'r', encoding='utf-8-sig') as ldafile:
        reader = csv.reader(ldafile)
        for line in reader:
            topic = [word.rstrip('\n') for word in line if word != '']
            t_list.append(topic)

    tmp1 = list()

    for t in t_list:
        tmp2 = t[1].split(' + ')
        for w in tmp2:
            tmp3 = w.split('*')
            tmp3[1] = tmp3[1].strip('"')
            if t[0] == '0': 
                tmp1.append(tmp3)
            else:
                tmp1[tmp2.index(w)] += tmp3

    with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav3/LDA/gen_'+year+'.csv', 'a+', newline = '', encoding='utf-8-sig') as ldafile:
        writer = csv.writer(ldafile)
        writer.writerows(tmp1)

In [45]:
#
t_list = list()
with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav3/LDA/gen_2011.csv', 'r', encoding='utf-8-sig') as ldafile:
    reader = csv.reader(ldafile)
    for line in reader:
        topic = [word.rstrip('\n') for word in line if word != '']
        t_list.append(topic)

tmp1 = list()

for t in t_list:
    tmp2 = t[1].split(' + ')
    for w in tmp2:
        tmp3 = w.split('*')
        tmp3[1] = tmp3[1].strip('"')
        if t[0] == '0': 
            tmp1.append(tmp3)
        else:
            tmp1[tmp2.index(w)] += tmp3
            
with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/datav3/LDA/gen_2011v2.csv', 'a+', newline = '', encoding='utf-8-sig') as ldafile:
    writer = csv.writer(ldafile)
    writer.writerows(tmp1)